## Program Design

### Folder Heirarchy

```
project
│   README.md
|   markdown-cheatsheet-online.pdf
│   HRTF_Estimation.ipynb    
|
│
└───Audio_devtest   #to test file iteration + processing not at-scale
│   │   file011.txt
|   │   file012.txt
│   └───MOS_test
│       │   file111.txt
│   
└───otherFolders
    │   file021.txt
```

### Logical Flow
1. Generate Database
    1. Generate Annotated Database (ambi_DB) of Ambisonic Representations of MongoDB Soundsources
    2. Obtain multiple HRTFs (HRTF_DB), annotated
    3. Create Secondary Annotated Database (main_DB) by convolving each in ambi_DB with each in HRTF_DB
2. Obtain MOS
    1. for each in main_DB
        1. Obtain 1 MOS for each in HRTF_DB
        2. Compare MOS values 
            a. search for hyperparameters that yield strong correlations
        3. Determine used HRTF for this from main_DB
3. Analysis + Discussion

In [ ]:
'''obtain_MOS notes / working thoughts:

--Before running the system, the input test signals are
assumed to be standardized/normalized to unitlength
and zero-mean, and the processed signal is
assumed to have been centered to zero-mean.
    ?? Is this still relevant? Need to test
    
--EN method requires HRIR, not SOFA



'''

## MOS Calculation

For Processed HRIR$(\alpha, \varphi)$ and test HRIR$(\theta,\phi)$

Transform each into freq domain (fft)
log scale (magnitude dB)

Compare each processed HRIR$(\alpha, \varphi)$ to every in test set HRIR$(\theta,\phi)$ 
via both Elastic-Net Regression and Jensen-Shannon Distance

### Elastic-Net Regression (EN)

$\hat{\beta} = argmin_\beta||\textbf{y}- \textbf{X}\beta||^2 + \delta||\beta||^2 + \lambda||\beta||_1$

where: 
- **y** is the processed signal, 
- **X** is a matrix where each column is one HRIR from test set HRIR(θ,ϕ)
- ????MAYBE $\beta$ is $(\theta, \phi)$
- "L2 norm" is defined as $||n||^2 = \sqrt{(|a|^2 + |b|^2)}$
    - where $n = (a, b)$ 
- δ is the L2 norm shrinkage parameter

- "L1 norm" is defined as $||n||_1 = |a| + |b|$
    - where $n = (a, b)$     
- λ is the L1 norm shrinkage parameter
    
and:
- $\hat{\beta}$ is the returned vector of coefficients, representing load on each predictor from test HRIR$(\theta,\phi)$ to reproduce the input response vector of HRIR$(\alpha, \varphi)$.

### Jenson-Shannon Distance (JSD)

Bounded and symmertical Kullback-Leibler (KL) divergence, measurement of similairty between two distributions. 

Computes the distance between the processed HRIR and all in test set. Lower values indivate statistically similar distributions, JSD of zero being identical. 

$JSD(P||T) = \sqrt{\frac{1}{2}[KL(P||\frac{P+T}{2}) + KL(T||\frac{P+T}{2})]}$

where: 
- $KL(P||T) = \sum(P(x)\frac{P(x)}{T(x)})$
- $P$ is the processed HRIR response
- $T$ is every member of Test HRIR set

### E-N distance computation

After computing EN distance between processed response signal HRIR$(\alpha, \varphi)$ and every member of the test set HRIR$(\theta,\phi)$, the returned $\hat{\beta}$ coefficients are indeced by \theta and \phi. 

Centroid + StdDev of $\hat{\beta}$ coefficients are of interest, as wel as angular distance from computed centroid to intended rendering position. Centroid must be calculated by first shifting distribution to center of image. 

**MOS-1** is the angular distance of computed centroid -> intended location

**MOS-2** is the std deviation of $\hat{\beta}$ coefficients as diffuse estimator

### JSD computation

JSD computed between processed HRIR$(\alpha, \varphi)$ and all in computed HRIR$(\theta,\phi)$, each value subtracted from 1 and plotted along azimuth + elevation axes. 

Angular Distance Map (ADM) is computed distance between intended location + every other location. ADM normalized between 0-1. 

**MOS-3** value at the index of ADM primary return index (highest coefficient value) of the JSD. Smaller MOS-3, more accurate rendering. 

**MOS-4** sum of all JSD coefficients multiplied by corresponding ADM values. Smaller MOS-4: more compact image. 

In [ ]:
def read_in_HRTF(folderpath, HRTF_type="wav")

    '''
    Reads in an HRTF from file into a 72-by-15 array of stereo impulse responses
    
    HRTF Ele Positions(15): -90, -75, -60, -45, -30, -25, -15, 0, 15, 25, 30, 45, 60, 75, 90
         Azi Positions(72): 0:355[::5]
    
    
    Parameters
    ----------
        folderpath (string) : 
        
        
        HRTF_type (string) : which type of HRTF file this function will look for.
        
        "WAV" will search for .wav files, located in subfolder ##K_##bit
            filenaming convention is "azi_#,#_ele_#,#.wav"
            
            ##K is Sample Rate
            ##bit is bit depth
            Commas represent decimals, prepend (-) if negative.
            e.g. 15 deg azi, -17.3 deg ele = "azi_15,0_ele_-17,3.wav"
        
        "sofa" will search for .sofa files
            filenaming convention is "D#_##K_##bit_###tap_FIR_SOFA.sofa"
            
            D# is HRTF subject no. (SADIE II database)
            ##K is Sample Rate
            ##bit is bit depth
            ###tap is FIR length in samples
            e.g. "D2_48K_24bit_256tap_FIR_SOFA.sofa"

    Returns
    -------
    
        HRTF_out (np.ndarray, shape(72,15)) : 
    '''
    
    

    pass

In [104]:
# def get_HRIR(HRTF, azi=0, ele=0, warnings=True):
def get_HRIR(azi=0, ele=0, warnings=True):
    
    '''Get the closest corresponding impulse response to 
    
    
    Parameters
    ----------
        HRTF   : np.array, shape(72, 15)
            HRTF to pull IR from
            
        azi : integer
            azimuth angle, either 0 < azi < 360 or -180 < azi < 180
            
        ele : integer
            elevation angle
            
        warnings : boolean
            allows for warnings printout
    
    
    Returns
    -------
    
        IR_out  :  np.array, shape(n, 2)
            Stereo Impulse response from chosen (or closest) azimuth and elevation
    
    '''
    # Flag to print out warning if input azi/ele didn't correspond with 
    closestChoiceFlag = False;
    negAziFlag = False;
    nearestAzi = 0;
    nearestEle = 0;
    
    # Check Azimuth input, convert to 0-359 if needed
    if (azi <= 0 and azi >=-180):
        negAzi = azi;
        negAziFlag = True; # Adjusts warning for legibility
        azi = azi + 360;
        
    elif (azi <= -180 or azi >= 360):
        print("Error: Azimuth must be between 0 and 360 or -180 and 180. Your input:", azi);
        return -1;
    elif (azi == 360):
        azi = 0;
    
    # Check Azimuth value against known HRIR azimuth angles
    if (azi % 5 == 0):
        print("we guccazi")
        chosenAzi = azi
        aziIdx = int(chosenAzi / 5)
    else:
        closestChoiceFlag = True;
        chosenAzi = ((azi - azi % 5)) + (round(azi % 5 / 5) * 5)
        aziIdx = int(chosenAzi / 5)
        
        
    # Check Elevation input, convert to 0-359 if needed
    if (ele <= -90 or ele >= 90):
        print("Error: Elevation must be between -90 and 90. Your input:", ele);
        return -1;
    
    # Check Azimuth value against known HRIR azimuth angles
    if (ele % 15 == 0):
        print("we guccele")
        chosenEle = ele
        eleIdx = int(chosenEle / 15);
    else:
        closestChoiceFlag = True;
        chosenEle = ((ele - ele % 15)) + (round(ele % 15 / 15) * 15)
        eleIdx = int(chosenEle / 15);
    
        
    
    print(aziIdx)
    print(eleIdx)
    
    # printout warning
    if (warnings == True and closestChoiceFlag == True):
        if (negAziFlag == True):
            print("Warning! Chosen Azi, Ele of [", negAzi, ele, "] do not match current HRTF set.",\
             "\nUsing nearest Azi, Ele fit of [", (chosenAzi - 360), chosenEle,\
             "]\n...To disable this warning, call function with 'warnings=False' ");
        else: 
            print("Warning! Chosen Azi, Ele of [", azi, ele, "] do not match current HRTF set.",\
             "\nUsing nearest Azi, Ele fit of [", chosenAzi, chosenEle,\
             "]\n...To disable this warning, call function with 'warnings=False' ");
    pass

In [109]:
import numpy as np
## Get_HRIR Testing
mock_HRTF = np.ones((72, 15))
print(mock_HRTF.shape)

get_HRIR(-56, 38)

(72, 15)
61
3
Warning! Chosen Azi, Ele of [ -56 38 ] do not match current HRTF set. 
Using nearest Azi, Ele fit of [ -55 45 ]
...To disable this warning, call function with 'warnings=False' 


In [106]:
def process_Elastic_net( ):
    
    '''
    β = argmin<sub>β</sub>
    
    
    Parameters
    ----------
     
        

    Returns
    -------
    
    
    '''

    
    
    pass

In [ ]:
def obtain_MOS (audiofile_path, HRTF_path):
    
    '''Get MOS (np.array) from Audiofile for Specific HRTF
    
    MOS = {MOS-1, MOS-2, MOS-3, MOS-4}
    
    Parameters
    ----------
    audiofile_path : str
        file name (*.wav) incl. path of stereo binaural signal
        
    HRTF_path: str
        HRTF folder including (*.wav) HRIRs for convolution
        
        

    Returns
    -------
    MOS : np.array, shape = (1, 4)
        outputs 4 MOS values for input audio file:
        
        MOS-1: (E-N) Localization Precision of Spectral Magnitude
        MOS-2: (E_N) Sptatial Variation / "Spread" of Spectral Magnitude
        MOS-3: (JSD) Localization Precision of Spectral Magnitude
        MOS-4: (JSD) Sptatial Variation / "Spread" of Spectral Magnitude
        

    
    
    
    '''


    
    pass
